In [2]:
# !pip install Box2D-py
# https://stackoverflow.com/questions/51811263/problems-pip-installing-box2d
# https://stackoverflow.com/questions/54027942/fatal-error-c1083-cannot-open-include-file-io-h-no-such-file-or-directory

#* Visual Studio C++ 14.0
#* Desktop development with C++
#* * MSVC build tools
#* * C++ CMake tools
#* * C++ AddressSanitizer
#* * Windows 11 SDK <-- important!



In [3]:
# !pip install gym --upgrade
# Update to gym version 0.24.0, which enables continuous bool in Car Racer
# Used to set action space as Discrete(5)

In [4]:
# !pip install gym[box2d]
#! DependencyNotInstalled: pygame is not installed, run `pip install gym[box2d]`

In [5]:
# https://www.gymlibrary.ml/environments/box2d/car_racing/

import gym

import numpy as np


c:\Users\jason\AppData\Local\Continuum\Anaconda3\lib\site-packages\gym\envs\registration.py:423: UserWarning: WARN: Custom namespace `ALE` is being overridden by namespace `ALE`. If you are developing a plugin you shouldn't specify a namespace in `register` calls. The namespace is specified through the entry point package metadata.
  logger.warn(


In [6]:
POP = 100
GENS = 8

EPISODES = 8

MUT_VAR = 0.1

MODEL_PATH = "model.bin"

ARCH = (96 ** 2, 24 ** 2, 5)

In [7]:
def grayscale(a):
    img = np.mean(a, axis=2)
    return img


def relu(a):
    return np.where(a > 0, a, 0)


def softmax(a):
    return np.exp(a) / np.sum(np.exp(a))


def time_elapsed(prev, cur, round_digit=8):
    delta_time = cur - prev
    return f"\tTime elapsed: {round(delta_time, round_digit)} seconds"


In [8]:
""" # Get observation space info and test grayscale
env = gym.make("CarRacing-v1", continuous=False)

print(env.action_space)

obs = env.reset()
# for row in obs:
    # print(row)
    
obs = grayscale(obs)

obs.resize((24, 24))

for row in obs: 
    print(row)
    
from matplotlib import pyplot as plt
plt.imshow(obs, interpolation='nearest')
plt.show()

env.close()
"""


' # Get observation space info and test grayscale\nenv = gym.make("CarRacing-v1", continuous=False)\n\nprint(env.action_space)\n\nobs = env.reset()\n# for row in obs:\n    # print(row)\n    \nobs = grayscale(obs)\n\nobs.resize((24, 24))\n\nfor row in obs: \n    print(row)\n    \nfrom matplotlib import pyplot as plt\nplt.imshow(obs, interpolation=\'nearest\')\nplt.show()\n\nenv.close()\n'

In [9]:
""" # Rendering to understand environment
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample() 
        obs, reward, done, info = env.step(action)
        score += reward

    print(f"Episode: {episode} Score: {score}")
env.close()
"""

' # Rendering to understand environment\nepisodes = 5\nfor episode in range(1, episodes+1):\n    obs = env.reset()\n    done = False\n    score = 0\n\n    while not done:\n        env.render()\n        action = env.action_space.sample() \n        obs, reward, done, info = env.step(action)\n        score += reward\n\n    print(f"Episode: {episode} Score: {score}")\nenv.close()\n'

In [10]:
class NN:
    def __init__(self, arch, net_to_copy=None, mut_var=MUT_VAR):
        if net_to_copy is None:
            weights = [
                np.random.rand(arch[1], arch[0]),
                np.random.rand(arch[2], arch[1]),
            ]

            biases = [
                np.zeros((arch[1],)),
                np.zeros((arch[2],)),
            ]

        else:
            weights = net_to_copy.weights
            biases = net_to_copy.biases

            weights = [
                weight + np.random.normal(0.0, mut_var, weight.shape)
                for weight in weights
            ]
            biases = [
                bias + np.random.normal(0.0, mut_var, bias.shape) for bias in biases
            ]

        # print([weight.shape for weight in weights])
        # print([bias.shape for bias in biases])

        self.weights = weights
        self.biases = biases

    def forward(self, _obs):
        weights = self.weights
        biases = self.biases

        _obs = np.array(_obs)

        hidden = relu(np.dot(weights[0], _obs) + biases[0])

        output = relu(np.dot(weights[1], hidden) + biases[1])

        probabilities = softmax(output)
        return np.argmax(probabilities)

    def evaluate(self, episodes=EPISODES, render_env=False, record=False):
        ep_rewards = []
        env = gym.make("CarRacing-v1", continuous=False)

        print(env.action_space)

        if record:
            env = gym.wrappers.Monitor(env, "recording")

        for i_ep in range(episodes):
            _obs = env.reset()
            done = False
            score = 0

            while not done:
                if render_env:
                    env.render()

                action = self.forward(grayscale(_obs).flatten())
                # print(action, env.action_space)

                _obs, reward, done, _ = env.step(action)
                score += reward

            ep_rewards.append(score)

        env.close()

        return np.array(ep_rewards).mean()


In [ ]:
nn = NN(ARCH)
nn.evaluate()